In [64]:
#Uncomment below to install animaloc
!pip install -e . --user

Obtaining file:///C:/Users/Machine%20Learning/Desktop/workspace-wildAI/HerdNet
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: animaloc
    Found existing installation: animaloc 0.2.0
    Uninstalling animaloc-0.2.0:
      Successfully uninstalled animaloc-0.2.0
  Running setup.py develop for animaloc


DEPRECATION: Loading egg at c:\users\machine learning\anaconda3\envs\herdnet\lib\site-packages\animaloc-0.2.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [65]:
# Imports
from pathlib import Path
import os
import random
import rasterio
from rasterio.windows import Window
import PIL
import torchvision
import numpy
import cv2
import skimage
from itertools import product
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from pathlib import Path
from albumentations import PadIfNeeded
import pandas as pd
from tqdm import tqdm
import shutil
from animaloc.data import ImageToPatches, PatchesBuffer, save_batch_images
import math

# Tiling data for annotation

### For GeoTiffs

In [ ]:
def save_patches(img_path:Path,dest_dir:Path,tilesize=512):

    # create directory
    if os.path.exists(dest_dir):
        shutil.rmtree(dest_dir)
        os.mkdir(dest_dir)
        print("emptying directory:", dest_dir)
    else:
        os.mkdir(dest_dir)
        print("creating directory:", dest_dir)
    
    # window reading with rasterio
    handler = rasterio.open(img_path)
    height, width = handler.meta['height'], handler.meta['width']
    coordinates = dict()
    count = 0
    for i,j in tqdm(product(list(range(0,height,tilesize)),list(range(0,width,tilesize)))):
        window = Window(j, i, tilesize, tilesize)
        
        try:
            chunk = handler.read(window=window)
            c,h,w = chunk.shape
            xmin, xmax = j, j+w
            ymin, ymax = i, i+h
            x_center = 0.5*(xmin+xmax)
            y_center = 0.5*(ymin+ymax)
            n_unique = np.unique(chunk).size
            if n_unique == 1:
                continue
            count += 1
            filename = img_path.name.split('.')[0] + f"-{j}-{i}.png"
            coordinates[count] = [xmin,xmax,ymin,ymax,x_center,y_center,filename]

            # save to disk
            chunk = np.transpose(chunk,(1,2,0))
            skimage.io.imsave(dest_dir/filename,chunk)          
                
        except Exception as e:
            print("Failed for",(i,j),e)
            pass

    cols = ['xmin','xmax','ymin','ymax','x_center','y_center','filename']
    coordinates = pd.DataFrame.from_dict(coordinates,
                                        orient='index',
                                        columns=cols)
    coordinates.to_csv(dest_dir/f"coordinates{img_path.name.split('.')[0]}.csv",index=False)
    handler.close()
    return coordinates

In [ ]:
# for datapath in Path("../annotation_data/camp6/").iterdir():
datapath = Path("../annotation_data/camp6/150m_RGB.tif")
dest_dir = datapath.parent/(datapath.name.split('.')[0])
# coordinates =  save_patches(datapath,dest_dir,tilesize=1024)

In [ ]:
coordinates

### For Images (jpg, rgb etc.)
Using Herdnet code

In [3]:
!pwd # current working dir

/home/ubuntu/workspace/HerdNet


#### Meanings of arguments
- ```-ratioheight``` : proportion of tile  w.r.t height of image. Example 0.5 means dividing the image in two bands w.r.t height. If it's 1.0 then this parameters is disabled.
- ```-ratiowidth``` : proportion of tile w.r.t to width of image. Example 0.9999 means the width of the tile is the same as the image. If it's 1.0 then this parameters is disabled.
- ```-overlapfactor``` : percentage of overlap. It should be less than 1.
- ```-rmheight``` : percentage of height to remove or crop at bottom and top
- ```-rmwidth``` : percentage of width to remove or crop on each side of the image
- ```-pattern``` : "**/*.JPG" will get all .JPG images in directory and subdirectories. On windows it will get both .JPG and .jpg. On unix it will only get .JPG images


In [ ]:
# Tiling Paul's data 
# Path(r"D:\PhD\Data per camp\Dry season\Kapiri\Farm-tiled").mkdir(exist_ok=True,parents=True)
!python ./tools/patcher.py "D:\PhD\Data per camp\Dry season\Kapiri August test\DJI_202310011649_001_Kapiri-6-7-8-55m" 0 0 0 -ratiowidth 0.34 -ratioheight 0.5 -dest "D:\PhD\Data per camp\Dry season\Kapiri August test\kapiri-6-7-8-55m_tiled"  -min 0.0 -all True -pattern "**/*.JPG"

In [ ]:
# Tiling Paul's data 
# Path(r"D:\PhD\Data per camp\Wet season\Kapiri\Camp 1\Rep 1 - tiled").mkdir(exist_ok=True,parents=True)
# !python ./tools/patcher.py "D:\PhD\Data per camp\Wet season\Kapiri\Camp 2\Rep 1" 0 0 0  -ratiowidth 1.0 -ratioheight 1.0 -rmheight 0.22 -rmwidth 0.28 -dest "D:\PhD\Data per camp\Wet season\Kapiri\Camp 2\Rep 1 - tiled"  -min 0.0 -all True -pattern "**/*.JPG"

In [ ]:
#uncomment to run
##!python ./tools/patcher.py "D:\PhD\Data per camp\Wet season\Kapiri\Camp 3\Rep 1" 0 0 0  -ratiowidth 1.0 -ratioheight 1.0 -rmheight 0.22 -rmwidth 0.28 -dest "D:\PhD\Data per camp\Wet season\Kapiri\Camp 3\Rep 1 - tiled"  -min 0.0 -all True -pattern "**/*.JPG"

In [ ]:
#uncomment to run
##!python ./tools/patcher.py "D:\PhD\Data per camp\Wet season\Kapiri\Camp 9_11\Rep 1" 0 0 0  -ratiowidth 0.5 -ratioheight 0.5 -rmheight 0.22 -rmwidth 0.28 -dest "D:\PhD\Data per camp\Wet season\Kapiri\Camp 9_11\Rep 1 - tiled"  -min 0.0 -all True -pattern "**/*.JPG"

In [10]:
#uncomment to run
##!python ./tools/patcher.py "D:\PhD\Data per camp\Wet season\Kapiri\Camp 4_5\Rep 1" 0 0 0  -ratiowidth 0.5 -ratioheight 0.99999 -rmheight 0.22 -rmwidth 0.28 -dest "D:\PhD\Data per camp\Wet season\Kapiri\Camp 4_5\Rep 1 - tiled"  -pattern "**/*.JPG"

In [ ]:
# New script for tiling data
!python ./tools/patcher.py "D:\PhD\Data per camp\Dry season\Kapiri\Camp 2\extra data\DJI_202310011415_004_Camp2" 0 0 0 -overlapfactor 0.2  -ratiowidth 0.5 -ratioheight 1.0 -rmheight 0.22 -rmwidth 0.28 -dest "D:\PhD\Data per camp\Dry season\Kapiri\Camp 2\extra data - tiled"  -pattern "**/*.JPG"

In [59]:
from itertools import product

def get_patches(image,image_width:int,image_height:int,tile_w:int,tile_h:int,overlaping_factor:float):
    patches = list()

    def get_coordinates():

        # x limits
        lim = math.ceil((image_width-tile_w)/((1-overlaping_factor)*tile_w))
        x_right = [math.floor(tile_w + i*(1-overlaping_factor)*tile_w) for i in range(lim)]
        x_coords = [(x-tile_w,x) for x in x_right]
        if len(x_coords)>0:
            left,right = x_coords[-1]
            x_coords[-1] = (left,image_width) # extending to remaining pixels

        # y limits
        lim = math.ceil((image_height-tile_h)/((1-overlaping_factor)*tile_h))
        y_bottom = [math.floor(tile_h + i*(1-overlaping_factor)*tile_h) for i in range(lim)]
        y_coords = [(y-tile_h,y) for y in y_bottom]
        if len(y_coords)>0:
            top,bottom = y_coords[-1]
            y_coords[-1] = (top,image_height) # extending to remaining pixels

        # tiles coordinates
        if len(y_coords)>0 and len(x_coords)>0:
            pass
        elif len(y_coords) == 0:
            y_coords = [(0,image_height),]
        elif len(x_coords) == 0:
            x_coords = [(0,image_width),]

        coordinates = product(x_coords,y_coords)
        return list(coordinates)
    
    coords =  get_coordinates()
    for (x_left,x_right),(y_top,y_bottom) in coords:
        patches.append(image[:,y_top:y_bottom,x_left:x_right])
    
    print(coords)
    
    return patches

img = np.zeros((3,1000,2000))
patches = get_patches(img,image_width=img.shape[2],image_height=img.shape[1],tile_w=450,tile_h=1000,overlaping_factor=0.2)

[((0, 450), (0, 1000)), ((360, 810), (0, 1000)), ((720, 1170), (0, 1000)), ((1080, 1530), (0, 1000)), ((1440, 2000), (0, 1000))]


In [60]:
[p.shape for p in patches]

[(3, 1000, 450),
 (3, 1000, 450),
 (3, 1000, 450),
 (3, 1000, 450),
 (3, 1000, 560)]

In [ ]:
# tiling validation data
Path("../general_dataset/val_splits").mkdir(exist_ok=True,parents=True)

!python ./tools/patcher.py ../general_dataset/val 640 640 100 \
    ../general_dataset/val_splits \
    -csv ../general_dataset/groundtruth/csv/val_big_size_A_B_E_K_WH_WB.csv \
    -min 0.0 -all False

In [ ]:
# tiling training data
Path("../general_dataset/train_splits").mkdir(exist_ok=True,parents=True)

!python ./tools/patcher.py ../general_dataset/train 640 640 100 \
    ../general_dataset/train_splits \
    -csv ../general_dataset/groundtruth/csv/train_big_size_A_B_E_K_WH_WB.csv \
    -min 0.0 -all False

In [ ]:
# tiling test data
Path("../general_dataset/test_splits").mkdir(exist_ok=True,parents=True)

!python ./tools/patcher.py ../general_dataset/test 640 640 100 \
    ../general_dataset/test_splits \
    -csv ../general_dataset/groundtruth/csv/test_big_size_A_B_E_K_WH_WB.csv \
    -min 0.0 -all False

In [6]:
# tiling Annotation data

# Destination of splits
Path("../EBP-Lindanda-cam0-splits").mkdir(exist_ok=True,parents=True)

!python ./tools/patcher.py ../EBP-Lindanda-cam0 512 512 64 \
    ../EBP-Lindanda-cam0-splits \
    -min 0.0 -all True

Exporting patches: 100%|██████████████████| 1226/1226 [1:02:52<00:00,  3.08s/it]


In [ ]:
# tiling data from Savmap dataset
# path_gt = '../savmap_dataset_v2/gt.csv'
# df_gt = pd.read_csv(path_gt)
# df_gt['labels'] = 0
# df_gt.rename(columns={'filename':'images',
#                       'xmin':'x_min',
#                       'xmax':'x_max',
#                       'ymin':'y_min',
#                       'ymax':'y_max'
#                       },inplace=True)

# df_gt.to_csv(path_gt,index=False,sep=',')

Path("../savmap_dataset_v2/train_splits").mkdir(exist_ok=True,parents=True)

!python ./tools/patcher.py ../savmap_dataset_v2/images 640 640 100 \
    ../savmap_dataset_v2/train_splits \
    -csv ../savmap_dataset_v2/gt.csv \
    -min 0.0 -all False

### Sampling emtpy and non empty images

In [ ]:
# sample training data

directory = Path(r"../savmap_dataset_v2/train/")
labels = directory/'labels'
dest_images = directory/'images_nonempty'
source_images = directory/'images'

# create destination images
dest_images.mkdir(exist_ok=True)

# move non empty images
# -- Uncomment to run > Be careful
# for file in labels.iterdir():
#     img_name = file.name.split('.')[0]+'.JPG'
#     if (source_images/img_name).exists():
#         os.rename(src=source_images/img_name,
#                 dst=dest_images/img_name)

# # and empty 
# -- Uncomment to run > Be careful!!
# num_non_empty = len(list(labels.iterdir()))
# num_empty_target = num_non_empty
# empty_images = list(source_images.iterdir())
# random.seed(41) # seeding for reproducibility
# random.shuffle(empty_images) # shuffle
# for file in empty_images[:num_empty_target]:
#     img_name = file.name
#     os.rename(src=file,
#                 dst=dest_images/img_name)

# rename folders
# os.rename(src=source_images,dst=directory/'images_empty')
# os.rename(src=dest_images,dst=directory/'images')


In [ ]:
len(list((directory/'images_empty').iterdir())),\
    len(list((directory/"images").iterdir())),\
        len(list((directory/"labels").iterdir()))


# Visualizing data

In [ ]:
import albumentations as A
from animaloc.datasets import CSVDataset
from animaloc.data.transforms import MultiTransformsWrapper, DownSample, PointsToMask, FIDT
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# patch_size = 640
# num_classes = 7
down_ratio = 1

val_dataset = CSVDataset(
    csv_file = '../wildlife_localizer_data/val/gt.csv',
    root_dir = '../wildlife_localizer_data/val',
    albu_transforms = [A.Normalize(p=1.0)],
    end_transforms = [DownSample(down_ratio=down_ratio, anno_type='bbox')]
    )

In [ ]:
val_dataset.anno_type

In [ ]:
labels = pd.read_csv('../wildlife_localizer_data/val/gt.csv')
labels.head()

In [ ]:
np.sort(labels['labels'].unique())

In [ ]:
# loader = DataLoader(dataset = val_dataset, batch_size = 1, shuffle = False)

In [ ]:
images, targets = val_dataset[0]

In [ ]:
targets

In [ ]:
# img = np.transpose(images.numpy(),(1,2,0))
# plt.imshow(img)

# Creating YOLO labels

In [ ]:
# Images dimensions
height,width = 640, 640

# Create label directory
for directory in Path("../wildlife_localizer_data/").iterdir():
    if not directory.is_dir():
        continue 
    labels_dir = Path(os.path.join(directory,'labels'))
    labels_dir.mkdir(exist_ok=True,parents=False) # create directory if it does not exist
    labels = pd.read_csv(os.path.join(directory,'gt.csv'))
    #-- Saving labels in YOLO format
    for img_filename,df_group in tqdm(labels.groupby(by='images'),desc=directory.name):
        df_group['width'] = (df_group['x_max'] - df_group['x_min'])/width 
        df_group['height'] = (df_group['y_max'] - df_group['y_min'])/height
        df_group['x'] = (0.5*(df_group['x_min'] + df_group['x_max']))/width # x center
        df_group['y'] = (0.5*(df_group['y_min'] + df_group['y_max']))/height # y center
        df_group['labels'] = 0 

        # print('\n\n',directory.name,'\n',df_group[['labels','x','y','width','height']])
        # break

        # uncomment to save labels files
        labels_filename     = img_filename.split('.')[0] + '.txt'
        if len(df_group)>0:
            cols = ['labels','x','y','width','height']
            df_group[cols].to_csv(os.path.join(labels_dir,labels_filename),
                                    sep=" ",
                                    header=False,
                                    index=False)


In [45]:
# for savmap data
directory = Path("../savmap_dataset_v2/train/")
labels_dir = Path(os.path.join(directory,'labels'))
labels_dir.mkdir(exist_ok=True,parents=False) # create directory if it does not exist
labels = pd.read_csv(os.path.join(directory,'gt.csv'))
#-- Saving labels in YOLO format
for img_filename,df_group in tqdm(labels.groupby(by='images'),desc=directory.name):
    df_group['width'] = (df_group['x_max'] - df_group['x_min'])/width 
    df_group['height'] = (df_group['y_max'] - df_group['y_min'])/height
    df_group['x'] = (0.5*(df_group['x_min'] + df_group['x_max']))/width # x center
    df_group['y'] = (0.5*(df_group['y_min'] + df_group['y_max']))/height # y center
    df_group['labels'] = 0 

    # print('\n\n',directory.name,'\n',df_group[['labels','x','y','width','height']])
    # break

    # uncomment to save labels files
    labels_filename = img_filename.split('.')[0] + '.txt'
    if len(df_group)>0:
        cols = ['labels','x','y','width','height']
        df_group[cols].to_csv(os.path.join(labels_dir,labels_filename),
                                    sep=" ",
                                    header=False,
                                    index=False)

train: 100%|██████████| 2750/2750 [00:09<00:00, 303.35it/s]


In [46]:
# Control splitting
num_missing = 0
num_found = 0
num_total = 0
for path in Path("../savmap_dataset_v2/train/images").iterdir():
    filename = path.name.split('.')[0]
    labelpath = Path("../savmap_dataset_v2/train/labels")/(filename + '.txt')
    num_total += 1
    if not labelpath.exists():
        num_missing += 1
    else:
        num_found += 1

num_missing,num_found,num_total

(30607, 2750, 33357)

In [ ]:
labels = pd.read_csv(os.path.join(directory,'gt.csv'))
labels['labels'].unique()

# Savmap data
Saving bounding boxes in VOC format

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
import torch 
from torchvision.utils import draw_bounding_boxes
from torchvision.ops import nms
from torchvision.transforms import PILToTensor
from PIL import Image

In [ ]:
annotations_path = Path("../savmap_dataset_v2/savmap_annotations_2014.shp")

In [ ]:
data = gpd.read_file(annotations_path)
data.head()

In [ ]:
# data.loc[data['IMAGEUUID']=='0a3ed15cfab4453795564140e8fde8ba']

In [7]:
# uuid = '0a3ed15cfab4453795564140e8fde8ba'
# polygons = data.loc[data['IMAGEUUID']==uuid,'geometry']
# polygons

In [ ]:
bboxes = dict()
count = 0
pil_to_tensor = PILToTensor()
for uuid in tqdm(np.unique(data.IMAGEUUID),desc='Getting bbox'):

    # identifier, filenmae w/o suffix
    uuid = str(uuid)

    # load img as tensor
    path_to_img = f"../savmap_dataset_v2/images/{uuid}.JPG"
    img_pil = Image.open(path_to_img)
    img_tensor = pil_to_tensor(img_pil)

    # get boxes
    polygons = data.loc[data['IMAGEUUID']==uuid,'geometry']
    boxes = np.array([list(polygon.bounds) for polygon in polygons])
    boxes = torch.from_numpy(boxes).float()

    # apply non max suppression o discard overlaping polygons
    areas = abs((boxes[:,2] - boxes[:,0])*(boxes[:,3] - boxes[:,1]))
    indices = nms(boxes=boxes,
                  scores= 1/areas, # discarding larger bbox when they overlap
                  iou_threshold=0.1)
    bbox = boxes[indices].numpy()

    # save bbox
    start, end = count, count+bbox.shape[0]
    for idx,i in enumerate(range(start, end)):
        bboxes[i] = [uuid,] + bbox[idx].tolist()
    count = end
# print('retained bbox indes:',indices)

In [ ]:
columns=['filename','x_min','y_min','x_max','y_max']
gt_bboxes = pd.DataFrame.from_dict(data=bboxes,
                       orient='index',
                       columns=columns)

for col in columns:
    if col != 'images':
        gt_bboxes[col] = gt_bboxes[col].apply(int)
    else:
        gt_bboxes[col] = gt_bboxes[col].apply(lambda x: f"{x}.JPG")

gt_bboxes['labels'] = 0 # class
gt_bboxes.head()

In [ ]:
# discarding invalid coordinates
# gt_bboxes.loc[gt_bboxes.min(axis=1,numeric_only=True)>=0]

In [ ]:
# Example: drawing bounding boxes

# load img as tensor
filename = gt_bboxes['filename'].sample(1).iloc[0]
path_to_img = f"../savmap_dataset/{filename}"
img_pil = Image.open(path_to_img)
img_tensor = PILToTensor()(img_pil)

boxes = gt_bboxes.loc[gt_bboxes['filename'] == filename, ['x_min','y_min','x_max','y_max']].to_numpy()
boxes = torch.from_numpy(boxes)

img_with_box = draw_bounding_boxes(img_tensor,
                                   boxes=boxes,
                                   colors="red",
                                   width=5).numpy().transpose((1,2,0))

plt.figure(figsize=(15,7))
plt.imshow(img_with_box)
plt.title(filename)
plt.show()

# Inference with YOLO

In [2]:
# Imports
from ultralytics import YOLO
import ultralytics
import torch
from pathlib import Path
from animaloc.data import ImageToPatches
from PIL import Image
import torch.nn.functional as F
from sahi import AutoDetectionModel
from sahi.models.yolov8 import Yolov8DetectionModel
from sahi.utils.cv import read_image
from sahi.predict import get_sliced_prediction, predict

In [2]:
import torch
torch.__version__

'2.2.2+cpu'

In [4]:
torch.cuda.is_available()

True

In [10]:
# load model
path_to_weights = Path(r"C:\Users\Machine Learning\Desktop\workspace-wildAI\yolov8.kaza.pt")

# path_to_weights = Path('../yolo-runs/exp4/weights/best.pt')



In [26]:
# model = YOLO('yolov8s.pt',task='detect').load(path_to_weights)

model = YOLO(path_to_weights,task='detect')


In [29]:
# model

In [6]:
# load image
dir_images = Path(r"D:\PhD\Data per camp\Dry season\Kapiri\Farm\DJI_202310040946_001_KapiriFarm1").glob('*.jpg')
img = Image.open(next(dir_images))

patcher = ImageToPatches(img,size=(640,640),overlap=0)
patches = patcher.make_patches()

# model.predict()
print(len(patcher))

117


In [70]:
# _ = patcher.show()

In [23]:
patches.shape

torch.Size([117, 3, 640, 640])

In [30]:
model = model.to('cuda')
preds = model(patches.to('cuda:0'))


0: 640x640 (no detections), 5.9ms
1: 640x640 (no detections), 5.9ms
2: 640x640 (no detections), 5.9ms
3: 640x640 (no detections), 5.9ms
4: 640x640 (no detections), 5.9ms
5: 640x640 (no detections), 5.9ms
6: 640x640 (no detections), 5.9ms
7: 640x640 (no detections), 5.9ms
8: 640x640 (no detections), 5.9ms
9: 640x640 (no detections), 5.9ms
10: 640x640 1 detection, 5.9ms
11: 640x640 1 detection, 5.9ms
12: 640x640 (no detections), 5.9ms
13: 640x640 (no detections), 5.9ms
14: 640x640 (no detections), 5.9ms
15: 640x640 (no detections), 5.9ms
16: 640x640 (no detections), 5.9ms
17: 640x640 (no detections), 5.9ms
18: 640x640 (no detections), 5.9ms
19: 640x640 (no detections), 5.9ms
20: 640x640 (no detections), 5.9ms
21: 640x640 (no detections), 5.9ms
22: 640x640 1 detection, 5.9ms
23: 640x640 (no detections), 5.9ms
24: 640x640 (no detections), 5.9ms
25: 640x640 (no detections), 5.9ms
26: 640x640 (no detections), 5.9ms
27: 640x640 (no detections), 5.9ms
28: 640x640 (no detections), 5.9ms
29: 64

In [23]:
# plt.imshow(patches[9].numpy().transpose(1,2,0))

In [31]:
# using sahi
# detection_model = AutoDetectionModel.from_pretrained(
#     model_type='yolov8',
#     model_path=path_to_weights,
#     image_size=640,
#     confidence_threshold=0.3,
#     device="cpu", # or 'cuda:0'
# )

detection_model = Yolov8DetectionModel(model=model,
                                    #    model_path=path_to_weights,
                                      confidence_threshold=0.4,
                                      device="cuda:0" # or 'cuda:0', 'cpu')
)
detection_model.device

device(type='cuda', index=0)

In [32]:
result = get_sliced_prediction(img, 
                               detection_model,
                               slice_height=640,
                               slice_width=640,
                               overlap_height_ratio=0.1,
                               overlap_width_ratio=0.1,
                               postprocess_type='NMS',
                               )

Performing prediction on 150 number of slices.


In [33]:
result.export_visuals(export_dir="../tmp/",
                      hide_labels=False,
                      hide_conf=True,text_size=3.)

In [21]:
result.to_coco_annotations()

[{'image_id': None,
  'bbox': [7637.795715332031,
   2068.1595458984375,
   170.678466796875,
   186.65167236328125],
  'score': 0.8245490789413452,
  'category_id': 0,
  'category_name': 'detection',
  'segmentation': [],
  'iscrowd': 0,
  'area': 31857},
 {'image_id': None,
  'bbox': [7377.245300292969,
   2517.1500854492188,
   277.61212158203125,
   178.13055419921875],
  'score': 0.7311072945594788,
  'category_id': 0,
  'category_name': 'detection',
  'segmentation': [],
  'iscrowd': 0,
  'area': 49451},
 {'image_id': None,
  'bbox': [7860.680755615234,
   2935.8993225097656,
   234.11785888671875,
   139.11868286132812],
  'score': 0.7070891261100769,
  'category_id': 0,
  'category_name': 'detection',
  'segmentation': [],
  'iscrowd': 0,
  'area': 32570},
 {'image_id': None,
  'bbox': [7156.594665527344,
   223.54080200195312,
   40.53228759765625,
   35.333099365234375],
  'score': 0.6239803433418274,
  'category_id': 0,
  'category_name': 'detection',
  'segmentation': [],
 

In [33]:
from urllib.parse import urlparse

In [35]:
# Parsing s3 url
url = 's3://wildaidata-test/p1/p2/000113a692ba61cd55ea3acb9c2f9c41709710a1_S2.JPG'
s3_img = Path(url)

img_path = s3_img
bucket = list(img_path.parents)[-3]
bucket_name = str(bucket).split('/')[-1]
filename = str(img_path).replace(f"{str(bucket)}/",'')

bucket_name, filename

('wildaidata-test', 'p1/p2/000113a692ba61cd55ea3acb9c2f9c41709710a1_S2.JPG')

In [37]:
r = urlparse(url, allow_fragments=False)
bucket_name = r.netloc
key = r.path.lstrip('/')
bucket_name,key

('wildaidata-test', 'p1/p2/000113a692ba61cd55ea3acb9c2f9c41709710a1_S2.JPG')

# Pre-annotate data for Label studio

In [2]:
from sahi.models.yolov8 import Yolov8DetectionModel
from sahi.predict import get_sliced_prediction, predict
from PIL import Image
import torch
from pathlib import Path
from tqdm import tqdm
import os
import json

In [17]:
def format_prediction(pred:dict,img_height:int,img_width:int):
        # formatting the prediction to work with Label studio
        x, y, width, height = pred['bbox']
        label = pred['category_name']
        score = pred['score']

        template = {
                    "from_name": "label",
                    "to_name": "image",
                    "type": "rectanglelabels",
                    'value': {
                        'rectanglelabels': [label],
                        'x': x / img_width * 100,
                        'y': y / img_height * 100,
                        'width': width / img_width * 100,
                        'height': height / img_height * 100
                    },
                    'score': score
        }

        return template

def format_path(file_path:Path,root_directory:Path=Path(r"D:")):
    relative_to_root = Path(os.path.relpath(file_path,root_directory)).as_posix()
    label_studio_format = f"/data/local-files/?d={root_directory.name}/{relative_to_root}"
    return label_studio_format


In [18]:
format_path(file_path=Path(r"D:\PhD\Data per camp\Extra training data\Bushriver_drone_paul\DJI_0004.JPG"),
            # root_directory=Path(r"D:\PhD\Data per camp")
            )

'/data/local-files/?d=/PhD/Data per camp/Extra training data/Bushriver_drone_paul/DJI_0004.JPG'

In [7]:
# Load detection model
detection_model = Yolov8DetectionModel(#model=model,
                                       model_path=r"C:\Users\Machine Learning\Desktop\workspace-wildAI\yolov8.kaza.pt",
                                      confidence_threshold=0.4,
                                      device="cuda" # or 'cuda:0', 'cpu')
)


In [56]:
# get pre-annotations 
# It assumes that all images have extension .JPG or .jpg
pre_annotations = list()
count = 0
data_directory = Path(r"D:\PhD\Data per camp\Dry season\Kapiri\Camp 1, 4, 5")
all_images = data_directory.glob("**/*.jpg",
                                 case_sensitive=False)
all_images = list(all_images)

for img_path in tqdm(all_images):
        count += 1
        # read file
        img = Image.open(img_path)
        # create template
        annotations = dict()
        annotations["id"] = count
        annotations["data"] = {"image":format_path(file_path=img_path)} 
        # get prediction on image
        result = get_sliced_prediction(img,       
                                        detection_model,
                                        slice_height=512,
                                        slice_width=512,
                                        overlap_height_ratio=0.1,
                                        overlap_width_ratio=0.1,
                                        postprocess_type='NMS',
                                        postprocess_match_metric='IOS',
                                        verbose=0
                                        )  
        img_height = result.image_height
        img_width = result.image_width
        formatted_pred = [format_prediction(pred,
                                                img_height=img_height,
                                                img_width=img_width) for pred in result.to_coco_annotations()]
        annotations["predictions"] = [{"model_version":"0.0.1",
                                       'result':formatted_pred
                                       }
                                       ]
        # store pre-annotations
        pre_annotations.append(annotations)


# save as json
save_path = "_".join([data_directory.name] + [parent.name for parent in data_directory.parents][:2]) + ".json"
save_path = Path(r"C:\Users\Machine Learning\Desktop\workspace-wildAI\Preannotated-data") / save_path
with open(save_path,"w") as file:
    json.dump(pre_annotations,file,indent=2)


100%|██████████| 2366/2366 [2:18:18<00:00,  3.51s/it]  


: 

In [55]:
len(all_images)

2061

In [44]:
# "-".join([] + [parent.name for parent in data_directory.parents][:2])
"_".join([data_directory.name] + [parent.name for parent in data_directory.parents][:2]) + ".json"

'Farm_Kapiri_Dry season.json'

In [30]:
# pre_annotations
len(all_images)

2061

In [31]:
# # save as json
# with open("./template_preannotations.json","w") as file:
#     json.dump(pre_annotations,file,indent=2)
